In [4]:
from __future__ import print_function

import numpy as np

from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam

# import data
# import metrics

誤差関数の定義

In [5]:
def triplet_loss(inputs):
    a, p, n = inputs
    p_dist = K.sum(K.square(a-p), axis=-1)
    n_dist = K.sum(K.square(a-n), axis=-1)
    return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)

In [7]:
imheight = 128
imwidth = 128
channels = 3
# ALPHA = 0.2
ALPHA=0.05

In [8]:
from keras.applications.vgg16 import VGG16
#include_top=false => Dense不要
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(imwidth, imheight, channels)), input_shape=None) 

15層目までは訓練しない

In [9]:
for layer in base_model.layers[:15]:
    layer.trainable=False

ネットワークを定義<br>

In [10]:
def create_embNet():
    x = base_model.output
    conv1 = Conv2D(32, (4,4) , padding='same', activation='relu')(x)
    conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(conv1)
    flatten = Flatten()(conv2) 
    dense_layer = Dense(20, activation='relu')(flatten)
    norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)
    return  Model(inputs=[base_model.input], outputs=norm_layer)

In [13]:
def build_model():
    a_in = Input(shape = (imheight, imwidth, channels), name='anchor_input')
    p_in = Input(shape = (imheight, imwidth, channels), name='positive_input')
    n_in = Input(shape = (imheight, imwidth, channels), name='negative_input')
    
    a_emb = create_embNet()(a_in)
    p_emb = create_embNet()(p_in)
    n_emb = create_embNet()(n_in)
    
    loss = merge(
        [a_emb,p_emb, n_emb],
        mode=triplet_loss,
        name='loss',
        output_shape=(1, ))
    
    model = Model(
        input=[a_in,p_in,n_in],
        output=loss)
    
    model.compile(loss=identity_loss, optimizer=Adam())
    
    return model

元々の
- 'crop_imgs/img/CLOTHING/T_Shirt'
- 'crop_imgs/img/TOPS/T_Shirt' <br>
を結合したので
- './T_Shirt_all/'<br>
のみを参照すれば良い

## './T_Shirt_all/train/'と'./T_Shirt_all/test/'に分けたい

In [16]:
import os

BASE_PATH = './T_Shirt_all/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

In [24]:
ids[0:5]

['id_00000001', 'id_00000019', 'id_00000051', 'id_00000058', 'id_00000068']

In [17]:
len(ids)

6377

In [18]:
sum = 0
for id_ in ids:
    files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
    sum += len(files)

In [19]:
sum

40115

In [20]:
def dirname2int(dirname):
    dirname = dirname[3::]
    return int(dirname)

In [ ]:
def get_triplets(BASE_PATH):
    triplets=[]
    # idの選択の仕方をランダムにしたらいいのか？
    for id_ in ids:
        files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
        con = sorted([x for x in files if 'comsumer' in x])
        shop = sorted([x for x in files if 'shop' in x ])
        combs = list(itertools.product(tuple(con),tuple(shop)))
        
        for comb in combs:
            comb = list(comb)
            neg_id = random.choice([x for x in ids if x != id_])
            neg_file = random.choice([BASE_PATH+neg_id+'/'+x for x in os.listdir(BASE_PATH+neg_id) if 'shop' in x])
            comb.append(neg_file)
            triplets.append(comb)
    
    return triplets

In [ ]:
BASE_PATH = 'crop_img/img/CLOTHING/T_Shirt/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

### 1epochのfitをfor文で回す
これによりnb_epochで指定するよりもコードに自由度が出る
- 1epochごとにtrainのアイテム集合からtripletを作成する。

In [ ]:
num_epochs = 100

In [ ]:
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = get_triplets(train)

    X = {
        'user_input': uid,
        'positive_item_input': pid,
        'negative_item_input': nid
    }

    model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              nb_epoch=1,
              verbose=0,
              shuffle=True)

    print('AUC %s' % metrics.full_auc(model, test))